In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13269,2024-03-14,Europa Euroliga,14:45,Olympiakos,Panathinaikos,1.67,2.28,151.5,1.83,1.95,-3.5,1.95,2.90,65ZGNr9M,0.598802,0.438596,0.546448,0.512821,0.037399,0.4,0.4,NaN,NaN,232.390,203.534799,0.875833,164.766,91.518569,0.555446,109.386,177.438,117.16,326.89,0.0,0.0,0.0,0.0,0.218398,0.044896,0.277011,0.65,0.130,5.153846,0.720887,0.7,-0.020887,0.32,0.064,20.000000,0.515665,0.5,-0.015665
13270,2024-03-14,Europa Euroliga,15:00,Crvena zvezda,Maccabi Tel Aviv,1.80,2.05,169.5,1.91,1.91,-2.5,2.00,2.30,Yiy9P4vA,0.555556,0.487805,0.523560,0.523560,0.043360,0.4,0.8,NaN,NaN,192.358,104.124901,0.541308,226.886,107.099828,0.472042,139.758,231.242,137.41,150.52,0.0,0.0,0.0,0.0,0.091832,0.000000,0.098666,-1.84,-0.368,-2.173913,0.600927,0.5,-0.100927,-3.58,-0.716,-1.466480,0.424868,0.3,-0.124868
13271,2024-03-14,Europa Euroliga,16:00,Alba Berlin,Monaco,5.40,1.18,161.5,1.91,1.91,9.5,2.00,1.03,SEzDOOgG,0.185185,0.847458,0.523560,0.523560,0.032643,0.4,0.0,NaN,NaN,367.906,251.459913,0.683490,138.086,29.517347,0.213761,186.910,165.302,212.29,186.34,0.0,0.0,0.0,0.0,0.906988,0.000000,0.452735,-2.36,-0.472,-9.322034,0.371418,0.3,-0.071418,1.50,0.300,0.600000,0.498400,0.5,0.001600
13272,2024-03-14,Europa Euroliga,16:30,Baskonia,Milano,1.61,2.42,161.5,1.91,1.91,-4.5,2.00,3.40,OtULM2OS,0.621118,0.413223,0.523560,0.523560,0.034341,0.4,0.6,NaN,NaN,186.974,100.750818,0.538849,171.578,42.865506,0.249831,132.474,188.670,113.74,111.65,0.0,0.0,0.0,0.0,0.284246,0.000000,0.366648,1.76,0.352,1.732955,0.574382,0.7,0.125618,-5.00,-1.000,-1.420000,0.389633,0.2,-0.189633
13273,2024-03-14,Austrália Liga Feminina Wnbl,07:30,Perth Lynx F,Southside F,1.40,2.85,173.5,1.85,1.89,-3.5,1.88,3.75,ShokierF,0.714286,0.350877,0.540541,0.529101,0.065163,0.8,0.6,NaN,NaN,171.520,52.668099,0.307067,141.682,18.590530,0.131213,148.632,133.810,197.64,144.18,0.0,0.0,0.0,0.0,0.482496,0.015125,0.469730,2.50,0.500,0.800000,0.651858,0.8,0.148142,0.21,0.042,44.047619,0.573337,0.6,0.026663
13274,2024-03-14,China Cba,08:35,Jilin,Guangzhou,2.41,1.54,208.5,1.86,1.88,1.5,1.97,1.53,xS6ouQ6f,0.414938,0.649351,0.537634,0.531915,0.064288,0.6,0.6,NaN,NaN,238.102,102.562051,0.430748,183.234,73.361707,0.400372,276.420,215.800,178.89,284.28,0.0,0.0,0.0,0.0,0.311485,0.007563,0.177787,-3.27,-0.654,-2.155963,0.394115,0.6,0.205885,-2.24,-0.448,-1.205357,0.508569,0.3,-0.208569
13275,2024-03-14,China Cba,08:35,Nanjing Tongxi,Qingdao,1.67,2.15,206.5,1.80,1.92,-4.5,2.05,2.53,f3Htt6il,0.598802,0.465116,0.555556,0.520833,0.063919,0.6,0.8,NaN,NaN,300.234,127.526483,0.424757,254.420,148.072054,0.581998,205.628,273.790,267.18,468.44,0.0,0.0,0.0,0.0,0.177702,0.045620,0.148215,4.57,0.914,0.733042,0.497568,0.7,0.202432,-1.01,-0.202,-5.693069,0.507961,0.5,-0.007961
13276,2024-03-14,Chipre Divisão A,14:30,Paeek,Apollon,2.31,1.57,150.5,1.84,1.84,-1.5,1.83,1.60,UwDdS5TF,0.432900,0.636943,0.543478,0.543478,0.069843,1.0,0.8,NaN,NaN,386.060,380.343986,0.985194,362.688,343.870703,0.948117,443.420,264.490,1048.00,594.00,1.0,1.0,0.0,0.0,0.269721,0.000000,0.094831,0.00,0.000,inf,0.000000,0.0,0.000000,-1.65,-0.330,-1.727273,0.000000,0.0,0.000000
13277,2024-03-14,Coreia Do Sul Kbl,07:00,Anyang JungKwanJang,Seoul Thunders,1.85,1.92,166.5,1.80,1.94,-1.5,1.95,2.03,YobY7mE1,0.540541,0.520833,0.555556,0.515464,0.061374,0.8,0.8,NaN,NaN,153.732,52.153442,0.339249,215.068,70.486454,0.327740,221.832,314.994,148.00,302.94,0.0,0.0,0.0,0.0,0.026259,0.052938,0.028426,-0.17,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:45,Europa Euroliga,Olympiakos,Panathinaikos,151.5,1.83,1.0000,Over
1,15:00,Europa Euroliga,Crvena zvezda,Maccabi Tel Aviv,169.5,1.91,1.0000,Over
2,16:30,Europa Euroliga,Baskonia,Milano,161.5,1.91,1.0000,Over
3,07:30,Austrália Liga Feminina Wnbl,Perth Lynx F,Southside F,173.5,1.85,1.0000,Over
4,08:35,China Cba,Nanjing Tongxi,Qingdao,206.5,1.80,1.0000,Over
5,07:00,Coreia Do Sul Kbl,Anyang JungKwanJang,Seoul Thunders,166.5,1.80,0.9761,Over
6,00:30,Eua Ncaa,Stephen F. Austin,Abilene Christian,139.5,1.80,1.0000,Over
7,00:30,Eua Ncaa,UC Santa Barbara,CS Northridge,150.5,1.80,1.0000,Over
8,13:30,Eua Ncaa,East Carolina,Tulsa,139.5,1.80,1.0000,Over
9,19:30,Eua Ncaa,Liberty,UTEP,134.5,1.80,1.0000,Over
